# Exploratory Data Analysis and Data Preprocessing

### Import Libraries

In [ ]:
import pandas as pd
import numpy as np
import folium
from folium.plugins import MarkerCluster
from math import sin, cos, sqrt, atan2, radians
from folium.plugins import MousePosition

The dataset contains the following fields:

- key - a unique identifier for each trip
- fare_amount - the cost of each trip in usd
- pickup_datetime - date and time when the meter was engaged
- passenger_count - the number of passengers in the vehicle (driver entered value)
- pickup_longitude - the longitude where the meter was engaged
- pickup_latitude - the latitude where the meter was engaged
- dropoff_longitude - the longitude where the meter was disengaged
- dropoff_latitude - the latitude where the meter was disengaged

## Define Relevant Functions

In [ ]:
def unique_values(dataframe):
    column_list = dataframe.columns.values.tolist()
    dict = {}
    for column in column_list:
        dict[column] = str(len(list(dataframe[str(column)].unique())))
        #print("Column: "+ column + " => " + str(len(list(dataframe[str(column)].unique()))))
    return pd.DataFrame.from_dict(dict,orient="index",columns=(["count"]))

def null_values(dataframe):
    column_list = dataframe.columns.values.tolist()
    dict = {}
    for column in column_list:
        if dataframe[str(column)].isnull().sum() > 0:
            dict[column] = dataframe[str(column)].isnull().sum()
            percentage_missing = dataframe[str(column)].isnull().sum()/len(dataframe)*100
            dict[column] = np.append(dict[column], percentage_missing)
    return pd.DataFrame.from_dict(dict,orient="index", columns = ['absolute', 'percentage'])

def data_types(df):
    return pd.DataFrame(df.dtypes.value_counts(),columns=(["count"]))

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

def map_locations(df,type):
    
    if type == "pickup":
        start_location = df.loc[0][["pickup_latitude","pickup_longitude"]]
        latitudes = df.pickup_latitude.tolist()
        longitudes = df.pickup_longitude.tolist()
        print("PICK - UP")
    else:
        start_location = df.loc[0][["dropoff_latitude","dropoff_longitude"]]
        latitudes = df.dropoff_latitude.tolist()
        longitudes = df.dropoff_longitude.tolist()
        print("DROP - OFF")
        

    locations = zip(latitudes,longitudes)
    
    marker_cluster = MarkerCluster()
    map = folium.Map(location=start_location, zoom_start=20)
    
    for marker in locations:
        Coordinates = [marker[0],marker[1]]
        marker = folium.Marker(Coordinates)
        marker_cluster.add_child(marker)
    

    formatter = "function(num) {return L.Util.formatNum(num, 5);};"
    mouse_position = MousePosition(
        position='topright',
        separator=' Long: ',
        empty_string='NaN',
        lng_first=False,
        num_digits=20,
        prefix='Lat:',
        lat_formatter=formatter,
        lng_formatter=formatter
    )

    map.add_child(mouse_position)
    map.add_child(marker_cluster)
    
    return map

## import Dataset

In [ ]:
df = pd.read_csv("uber.csv")
df.drop(["Unnamed: 0","key"],axis=1,inplace=True) # Columns are not relevant to any prediction
df.head()

## Create Datetime Objects

In [ ]:
df.pickup_datetime = pd.to_datetime(df["pickup_datetime"])
df["year"] = df.pickup_datetime.apply(lambda x: x.year)
df["month"] = df.pickup_datetime.apply(lambda x: x.month)
df["day"] = df.pickup_datetime.apply(lambda x: x.day)
df["hour"] = df.pickup_datetime.apply(lambda x: x.hour)
df["minute"] = df.pickup_datetime.apply(lambda x: x.minute)
df["second"] = df.pickup_datetime.apply(lambda x: x.second)

df["date"] = df.pickup_datetime.apply(lambda x: x.date)
df["time"] = df.pickup_datetime.apply(lambda x: x.time)
df["day_of_week"] = df.pickup_datetime.apply(lambda x: x.weekday())


df.dropna(inplace=True)
df.drop("pickup_datetime",axis=1,inplace=True)

## Data Overview

In [ ]:
df.head()

In [ ]:
data_types(df)

In [ ]:
unique_values(df)

In [ ]:
null_values(df)

## Show Original Overview of Pick-up Locations using a Map view

In [ ]:
map = map_locations(df,"pickup")

In [ ]:
map

### From the above map:
- It is likely that the above dataset primarily consists of uber rides in NYC
- Several Pick-up Locations are at unlikely locations (e.g. in the ocean) => Likely Errors
- Only Uber rides that start in NYC and have a plausible Destination shall be considered for this project

## Filtering Out Non-Relevant Locations

In [ ]:
# Boundary Coordinates
min_latitude = 40.50
max_latitude = 41.20
min_longitude = -74.55
max_longitude = -73.05

# Define Filtered Dataframe
df1 = df[(df["pickup_latitude"] > min_latitude) & 
         (df["pickup_latitude"] < max_latitude) & 
         (df["pickup_longitude"] > min_longitude) & 
         (df["pickup_longitude"] < max_longitude) &
         (df["dropoff_latitude"] > min_latitude) & 
         (df["dropoff_latitude"] < max_latitude) & 
         (df["dropoff_longitude"] > min_longitude) & 
         (df["dropoff_longitude"] < max_longitude)  
        ].reset_index().drop("index",axis=1)

## Create Distance Column

In [ ]:
distance_list = []

for i in range(len(df1)):
    pickup = df1.loc[i][["pickup_latitude","pickup_longitude"]].tolist()
    dropoff = df1.loc[i][["dropoff_latitude","dropoff_longitude"]].tolist()
    distance = calculate_distance(pickup[0], pickup[1], dropoff[0], dropoff[1])
    distance_list.append(distance)

In [ ]:
df1["distance"] = distance_list

In [ ]:
df1.head()

## Save Dataframe

In [ ]:
df1.to_csv("uber_preprocessed.csv")